In [2]:
!pip install -U langchain-huggingface
!pip install \
    langchain_community \
    langchain_pinecone \
    langchain_openai \
    unstructured \
    langchain-text-splitters
!pip -q install langchain huggingface_hub transformers sentence_transformers
!pip install langchain_huggingface
!pip install pinecone-client -q
!pip install langchain

!pip install --upgrade langchain openai -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!pip install transformers sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 24.3 MB/s eta 0:00:00
   ━━━━

In [3]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import DirectoryLoader #1
from langchain_text_splitters import RecursiveCharacterTextSplitter #2
import os
import glob
from sentence_transformers import SentenceTransformer
from langchain_community.llms import HuggingFaceHub

In [4]:
from google.colab import userdata
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')
os.environ['HUGGINGFACE_API_TOKEN']=userdata.get('HUGGINGFACE_API_TOKEN')

1. Yoga For Mood Boosting Power? Elevate Your Mood with These Yoga Poses
by RN To Zen Staff | Mental Health, Yoga

2. Yoga for a Happy Mood
PIOR Living / Yoga Life / Yoga for a Happy Mood

3. Study: Body posture affects confidence in your own thoughts
Published: Monday, October 5, 2009 - 11:38 in Psychology & Sociology

4. Winter Blues? 7 yoga poses to uplift mood and combat seasonal depression
TOI Lifestyle Desk / etimes.in / Dec 4, 2024, 16:00 IST

5. Building bridges from Baby Bloomers to Gen Z and how Yoga YV can help

In [5]:
directory = "/content/data"

def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)
len(documents)

1

In [20]:
def split_docs(documents, chunk_size=800, chunk_overlap=80):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
len(docs)


180

In [21]:
docs[0].page_content

'In the hustle and bustle of daily life, we often find ourselves searching for moments of calm and happiness. One effective way to elevate your mood and promote emotional well-being is through the practice of yoga. Yoga offers a unique blend of physical postures, mindful breathing, and mental focus that can leave you feeling rejuvenated and uplifted. In this blog post, we’ll explore a selection of yoga poses specifically designed to boost your mood and bring a sense of positivity into your day.\n\nyoga for mood\n\nThe Power of Yoga for Mood Enhancement:'

In [22]:
# split our document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

In [23]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="neuml/pubmedbert-base-embeddings")

In [24]:
query_result = embeddings.embed_query("I feel so overwhelmed and alone")
len(query_result)

768

In [25]:
from langchain_pinecone import PineconeVectorStore
index_name = "yoga"
vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [26]:
query = "I feel so overwhelmed and alone"
similar_docs = vectorstore.similarity_search(query)
similar_docs

[Document(id='ca508df8-04e4-4bbf-b4cd-63526563fff0', metadata={'source': '/content/data/yoga.docx'}, page_content='connect with friends or loved ones about what you’re going through. I know it’s hard. Maybe you don’t want to be a bother, or you just don’t have the energy for a heavy conversation but it does lift a weight off you should you decide to let it out. The first time I came out–more like exploded in tears about my depression was with my partner. It helped more than I could imagine. It also became easier to talk about it with other people in my life. They showed me Apps, took me out for walks, and showed me that they wanted to connect. I have taken a few choice persons up on their offers but I’m not ready to see more than 1 person at a time. I think I’ll get there though.'),
 Document(id='5b6def59-ac49-46cb-b864-ac16e5e8444e', metadata={'source': '/content/data/yoga.docx'}, page_content='In my case, SAD has affected me more and more the longer I live in Europe. Being Filipina, 

In [27]:
from langchain_community.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_TOKEN"),
    repo_id=repo_id,
    model_kwargs={"temperature": 0.7, "max_new_tokens": 500},
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)

{'query': 'I feel so overwhelmed and alone',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nconnect with friends or loved ones about what you’re going through. I know it’s hard. Maybe you don’t want to be a bother, or you just don’t have the energy for a heavy conversation but it does lift a weight off you should you decide to let it out. The first time I came out–more like exploded in tears about my depression was with my partner. It helped more than I could imagine. It also became easier to talk about it with other people in my life. They showed me Apps, took me out for walks, and showed me that they wanted to connect. I have taken a few choice persons up on their offers but I’m not ready to see more than 1 person at a time. I think I’ll get there though.\n\nIn my case, SAD has affected me more and more the longer I live in Europe. Being Filipina, I lived